## PA2

In [1]:
'''
@author: jcheung

Developed for Python 2. Automatically converted to Python 3; may result in bugs.
'''
import xml.etree.cElementTree as ET
import codecs

class WSDInstance:
    def __init__(self, my_id, lemma, context, index):
        self.id = my_id         # id of the WSD instance
        self.lemma = lemma      # lemma of the word whose sense is to be resolved
        self.context = context  # lemma of all the words in the sentential context
        self.index = index      # index of lemma within the context
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%s\t%s\t%s\t%d' % (self.id, self.lemma, ' '.join(self.context), self.index)

def load_instances(f):
    '''
    Load two lists of cases to perform WSD on. The structure that is returned is a dict, where
    the keys are the ids, and the values are instances of WSDInstance.
    '''
    tree = ET.parse(f)
    root = tree.getroot()

    dev_instances = {}
    test_instances = {}

    for text in root:
        if text.attrib['id'].startswith('d001'):
            instances = dev_instances
        else:
            instances = test_instances
        for sentence in text:
            # construct sentence context
            context = [to_ascii(el.attrib['lemma']) for el in sentence]
            for i, el in enumerate(sentence):
                if el.tag == 'instance':
                    my_id = el.attrib['id']
                    lemma = to_ascii(el.attrib['lemma'])
                    instances[my_id] = WSDInstance(my_id, lemma, context, i)
    return dev_instances, test_instances

def load_key(f):
    '''
    Load the solutions as dicts.
    Key is the id
    Value is the list of correct sense keys.
    '''
    dev_key = {}
    test_key = {}
    for line in open(f):
        if len(line) <= 1: continue
        #print (line)
        doc, my_id, sense_key = line.strip().split(' ', 2)
        if doc == 'd001':
            dev_key[my_id] = sense_key.split()
        else:
            test_key[my_id] = sense_key.split()
    return dev_key, test_key

def to_ascii(s):
    # Remove all non-ASCII characters and ensure the result is a Unicode string
    return codecs.encode(s, 'ascii', 'ignore').decode('ascii')


In [2]:
if __name__ == '__main__':
    data_f = 'multilingual-all-words.en.xml'
    key_f = 'wordnet.en.key'
    dev_instances, test_instances = load_instances(data_f)
    dev_key, test_key = load_key(key_f)

    # IMPORTANT: keys contain fewer entries than the instances; need to remove them
    dev_instances = {k:v for (k,v) in dev_instances.items() if k in dev_key}
    test_instances = {k:v for (k,v) in test_instances.items() if k in test_key}

    # read to use here
    print(len(dev_instances)) # number of dev instances
    print(len(test_instances)) # number of test instances


194
1450


In [3]:
# Import necessary NLTK and other libraries
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.wsd import lesk
from nltk import word_tokenize, pos_tag
from transformers import BertTokenizer, BertModel
import torch

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
def evaluate(predictions, gold):
    '''
    Evaluate the predictions against the gold standard.
    '''
    total = 0
    correct = 0
    for inst_id, sense_keys in gold.items():
        total += 1
        predicted_key = predictions.get(inst_id)
        if predicted_key is None:
            continue  # No prediction
        if predicted_key in sense_keys:
            correct += 1
    accuracy = correct / total
    return accuracy


#### Most frequent sense and LESK

In [5]:
def preprocess(text):
    '''
    Preprocess text by tokenizing, lowercasing, removing stop words, and lemmatizing.
    '''
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return set(tokens)

def most_frequent_sense(instances):
    '''
    Implement the most frequent sense baseline.
    '''
    predictions = {}
    for inst_id, instance in instances.items():
        lemma = instance.lemma
        # Get synsets for the lemma
        synsets = wn.synsets(lemma)
        if synsets:
            synset = synsets[0]  # Most frequent sense
            # Get sense key corresponding to the lemma
            sense_keys = [l.key() for l in synset.lemmas() if l.name() == lemma]
            if sense_keys:
                predictions[inst_id] = sense_keys[0]
            else:
                predictions[inst_id] = synset.lemmas()[0].key()
        else:
            predictions[inst_id] = None  # No prediction
    return predictions

def lesk_algorithm(instances):
    '''
    Implement NLTK's Lesk algorithm.
    '''
    predictions = {}
    for inst_id, instance in instances.items():
        context_sentence = ' '.join(instance.context)
        target_word = instance.context[instance.index]
        # Apply Lesk algorithm
        synset = lesk(instance.context, target_word)
        if synset:
            sense_keys = [l.key() for l in synset.lemmas() if l.name() == target_word]
            if sense_keys:
                predictions[inst_id] = sense_keys[0]
            else:
                predictions[inst_id] = synset.lemmas()[0].key()
        else:
            predictions[inst_id] = None
    return predictions


In [6]:
# Evaluate Most Frequent Sense Baseline on Dev Set
dev_mfs_predictions = most_frequent_sense(dev_instances)
dev_mfs_accuracy = evaluate(dev_mfs_predictions, dev_key)
print('Dev Most Frequent Sense Baseline Accuracy:', dev_mfs_accuracy)

# Evaluate Lesk Algorithm on Dev Set
dev_lesk_predictions = lesk_algorithm(dev_instances)
dev_lesk_accuracy = evaluate(dev_lesk_predictions, dev_key)
print('Dev Lesk Algorithm Accuracy:', dev_lesk_accuracy)

# Similarly evaluate on Test Set
test_mfs_predictions = most_frequent_sense(test_instances)
test_mfs_accuracy = evaluate(test_mfs_predictions, test_key)
print('Test Most Frequent Sense Baseline Accuracy:', test_mfs_accuracy)

test_lesk_predictions = lesk_algorithm(test_instances)
test_lesk_accuracy = evaluate(test_lesk_predictions, test_key)
print('Test Lesk Algorithm Accuracy:', test_lesk_accuracy)


Dev Most Frequent Sense Baseline Accuracy: 0.6752577319587629
Dev Lesk Algorithm Accuracy: 0.32989690721649484
Test Most Frequent Sense Baseline Accuracy: 0.6172413793103448
Test Lesk Algorithm Accuracy: 0.33655172413793105


**SAMPLE OUTPUT**

In [7]:
# Extract the first instance from dev_instances
first_instance_id = next(iter(dev_instances))
first_instance = {first_instance_id: dev_instances[first_instance_id]}
print(first_instance.items())

# Test the most_frequent_sense function
single_mfs_prediction = most_frequent_sense(first_instance)
print("Single Instance MFS Prediction:", single_mfs_prediction)

# Test the lesk_algorithm function
single_lesk_prediction = lesk_algorithm(first_instance)
print("Single Instance Lesk Prediction:", single_lesk_prediction)


dict_items([('d001.s001.t002', <__main__.WSDInstance object at 0x7dd58196edd0>)])
Single Instance MFS Prediction: {'d001.s001.t002': 'group%1:03:00::'}
Single Instance Lesk Prediction: {'d001.s001.t002': 'group%2:33:00::'}


#### BERT

In [ ]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def bert_wsd(instances):
    '''
    Implement WSD using a pretrained BERT model.
    '''
    predictions = {}
    for inst_id, instance in instances.items():
        context_sentence = ' '.join(instance.context)
        # Get embedding for the context sentence
        inputs = tokenizer(context_sentence, return_tensors='pt')
        outputs = model(**inputs)
        context_embedding = outputs.last_hidden_state.mean(1)

        # Get synsets for the lemma
        synsets = wn.synsets(instance.lemma)
        max_similarity = -float('inf')
        best_sense = None
        for synset in synsets:
            definition = synset.definition()
            # Get embedding for the sense definition
            inputs = tokenizer(definition, return_tensors='pt')
            outputs = model(**inputs)
            definition_embedding = outputs.last_hidden_state.mean(1)
            # Compute cosine similarity
            similarity = torch.cosine_similarity(context_embedding, definition_embedding)
            if similarity > max_similarity:
                max_similarity = similarity
                best_sense = synset
        if best_sense:
            sense_keys = [l.key() for l in best_sense.lemmas() if l.name() == instance.lemma]
            if sense_keys:
                predictions[inst_id] = sense_keys[0]
            else:
                predictions[inst_id] = best_sense.lemmas()[0].key()
        else:
            predictions[inst_id] = None
    return predictions


In [ ]:
# Evaluate BERT WSD on Dev Set
dev_bert_predictions = bert_wsd(dev_instances)
dev_bert_accuracy = evaluate(dev_bert_predictions, dev_key)
print('Dev BERT WSD Accuracy:', dev_bert_accuracy)

# Evaluate BERT WSD on test Set
test_bert_predictions = bert_wsd(test_instances)
test_bert_accuracy = evaluate(test_bert_predictions, test_key)
print('Test BERT WSD Accuracy:', test_bert_accuracy)


Dev BERT WSD Accuracy: 0.520618556701031
Test BERT WSD Accuracy: 0.46620689655172415


#### Improved bert

In [ ]:
# Import necessary libraries
from transformers import BertTokenizerFast, BertModel
import torch

# Initialize BERT tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode

def bert_wsd_improved(instances):
    '''
    Improved BERT WSD implementation that extracts the embedding of the target word
    and enriches sense representations.
    '''
    predictions = {}
    for inst_id, instance in instances.items():
        context_sentence = ' '.join(instance.context)
        target_word = instance.context[instance.index]
        lemma = instance.lemma.lower()

        # Tokenize the context sentence with offsets
        inputs = tokenizer(context_sentence, return_tensors='pt', return_offsets_mapping=True, truncation=True)
        offset_mapping = inputs.pop('offset_mapping')[0]
        input_ids = inputs['input_ids'][0]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # Find the character positions of each token
        offsets = offset_mapping.numpy()

        # Find the character positions of the target word in the original sentence
        target_word_start = sum(len(w) + 1 for w in instance.context[:instance.index])  # +1 for spaces
        target_word_end = target_word_start + len(target_word)

        # Identify token indices corresponding to the target word
        target_token_indices = []
        for idx, (start, end) in enumerate(offsets):
            if start <= target_word_end and end >= target_word_start:
                target_token_indices.append(idx)

        if not target_token_indices:
            # If target word not found, skip this instance
            predictions[inst_id] = None
            continue

        # Get embeddings for the target word tokens
        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.last_hidden_state.squeeze(0)
            target_embeddings = hidden_states[target_token_indices]
            # Average embeddings if multiple tokens
            context_embedding = target_embeddings.mean(0).unsqueeze(0)

        # Get synsets for the lemma
        synsets = wn.synsets(lemma)
        if not synsets:
            predictions[inst_id] = None
            continue
        max_similarity = -float('inf')
        best_sense = None
        for synset in synsets:
            # Create sense representation by combining definition and examples
            sense_text = synset.definition() + ' ' + ' '.join(synset.examples())
            # Tokenize sense text
            inputs_def = tokenizer(sense_text, return_tensors='pt', truncation=True)
            with torch.no_grad():
                outputs_def = model(**inputs_def)
                definition_embedding = outputs_def.last_hidden_state.mean(1)
            # Compute cosine similarity
            similarity = torch.nn.functional.cosine_similarity(context_embedding, definition_embedding)
            if similarity.item() > max_similarity:
                max_similarity = similarity.item()
                best_sense = synset
        if best_sense:
            # Get the sense key
            sense_keys = [l.key() for l in best_sense.lemmas() if l.name().lower() == lemma]
            if sense_keys:
                predictions[inst_id] = sense_keys[0]
            else:
                predictions[inst_id] = best_sense.lemmas()[0].key()
        else:
            predictions[inst_id] = None
    return predictions

# Evaluate Improved BERT WSD on Dev Set
dev_bert_predictions = bert_wsd_improved(dev_instances)
dev_bert_accuracy = evaluate(dev_bert_predictions, dev_key)
print('Dev Improved BERT WSD Accuracy:', dev_bert_accuracy)

# Evaluate Improved BERT WSD on Test Set
test_bert_predictions = bert_wsd_improved(test_instances)
test_bert_accuracy = evaluate(test_bert_predictions, test_key)
print('Test Improved BERT WSD Accuracy:', test_bert_accuracy)


Dev Improved BERT WSD Accuracy: 0.5773195876288659
Test Improved BERT WSD Accuracy: 0.5524137931034483


#### Enhanced Lesk Algorithm

In [10]:
def enhanced_lesk(instances):
    '''
    Implement an enhanced Lesk algorithm that expands the sense definitions.
    '''
    predictions = {}
    for inst_id, instance in instances.items():
        # Preprocess context
        context = preprocess(' '.join(instance.context))
        synsets = wn.synsets(instance.lemma)
        max_overlap = 0
        best_sense = None
        for synset in synsets:
            signature = set()
            # Include definition and examples
            signature.update(preprocess(synset.definition()))
            for example in synset.examples():
                signature.update(preprocess(example))
            # Include hypernyms and hyponyms definitions
            for related_synset in synset.hypernyms() + synset.hyponyms():
                signature.update(preprocess(related_synset.definition()))
            # Compute overlap
            overlap = len(context.intersection(signature))
            if overlap > max_overlap:
                max_overlap = overlap
                best_sense = synset
        if best_sense:
            sense_keys = [l.key() for l in best_sense.lemmas() if l.name() == instance.lemma]
            if sense_keys:
                predictions[inst_id] = sense_keys[0]
            else:
                predictions[inst_id] = best_sense.lemmas()[0].key()
        else:
            predictions[inst_id] = None
    return predictions

In [11]:
# Evaluate Enhanced Lesk Algorithm on Dev Set
dev_enhanced_predictions = enhanced_lesk(dev_instances)
dev_enhanced_accuracy = evaluate(dev_enhanced_predictions, dev_key)
print('Dev Enhanced Lesk Accuracy:', dev_enhanced_accuracy)

# Evaluate Enhanced Lesk Algorithm on Dev Set
test_enhanced_predictions = enhanced_lesk(test_instances)
test_enhanced_accuracy = evaluate(test_enhanced_predictions, test_key)
print('Test Enhanced Lesk Accuracy:', test_enhanced_accuracy)


Dev Enhanced Lesk Accuracy: 0.4072164948453608
Test Enhanced Lesk Accuracy: 0.46206896551724136


In [12]:
# Extract the first instance from dev_instances
first_instance_id = next(iter(dev_instances))
first_instance = {first_instance_id: dev_instances[first_instance_id]}
print(first_instance.items())

# Test the enhanced_lesk function
single_enhanced_lesk_prediction = enhanced_lesk(first_instance)
print("Single Instance enhanced_lesk Prediction:", single_enhanced_lesk_prediction)


dict_items([('d001.s001.t002', <__main__.WSDInstance object at 0x7dd58196edd0>)])
Single Instance enhanced_lesk Prediction: {'d001.s001.t002': 'group%1:03:00::'}


#### yarowsky

In [8]:
def yarowsky_algorithm(dev_instances, dev_key, k=10, iterations=10):
    '''
    Yarowsky's algorithm for Word Sense Disambiguation.
    - dev_instances: dictionary with the development instances
    - dev_key: dictionary with the correct senses for the dev instances
    - k: number of initial seeds (senses)
    - iterations: number of iterations for spreading the seeds
    '''
    # 1. Initial seed set based on correct senses in the dev set
    seeds = {}
    for my_id, instance in dev_instances.items():
        if my_id in dev_key:
            correct_senses = dev_key[my_id]
            for sense in correct_senses:
                if sense not in seeds:
                    seeds[sense] = set()
                seeds[sense].add(my_id)

    # 2. Iterative sense propagation based on context similarity
    for iteration in range(iterations):
        new_seeds = {sense: set(seeds[sense]) for sense in seeds}

        for my_id, instance in dev_instances.items():
            if my_id in dev_key:
                continue  # Skip the labeled instances in dev set
            context = set(instance.context)

            # Find most similar sense based on context overlap
            best_sense = None
            best_overlap = 0
            for sense, seed_instances in seeds.items():
                overlap = len(context.intersection(*[set(dev_instances[id].context) for id in seed_instances]))
                if overlap > best_overlap:
                    best_sense = sense
                    best_overlap = overlap

            if best_sense:
                new_seeds[best_sense].add(my_id)

        seeds = new_seeds

    # Assign sense to instances based on seeds
    predicted_senses = {}
    for my_id, instance in dev_instances.items():
        best_sense = None
        best_overlap = 0
        for sense, seed_instances in seeds.items():
            overlap = len(set(instance.context).intersection(*[set(dev_instances[id].context) for id in seed_instances]))
            if overlap > best_overlap:
                best_sense = sense
                best_overlap = overlap
        predicted_senses[my_id] = best_sense

    return predicted_senses


In [9]:
def calculate_accuracy(predicted_senses, gold_key):
    '''
    Calculate accuracy by comparing predicted senses to the gold key.
    - predicted_senses: dictionary {id: sense}
    - gold_key: dictionary {id: [list of correct senses]}
    Returns:
        - Accuracy as a float.
    '''
    correct = 0
    total = len(predicted_senses)

    for my_id, predicted_sense in predicted_senses.items():
        if my_id in gold_key and predicted_sense in gold_key[my_id]:
            correct += 1

    return correct / total if total > 0 else 0.0

if __name__ == '__main__':
    # Apply Yarowsky's algorithm
    predicted_dev_senses = yarowsky_algorithm(dev_instances, dev_key, k=10, iterations=10)
    predicted_test_senses = yarowsky_algorithm(test_instances, test_key, k=10, iterations=10)

    # Calculate accuracies
    dev_accuracy = calculate_accuracy(predicted_dev_senses, dev_key)
    test_accuracy = calculate_accuracy(predicted_test_senses, test_key)

    print(f'Development Accuracy: {dev_accuracy * 100:.2f}%')
    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Development Accuracy: 14.43%
Test Accuracy: 14.28%


In [13]:
# Extract the first instance from dev_instances
first_instance_id = next(iter(dev_instances))
first_instance = {first_instance_id: dev_instances[first_instance_id]}
print(first_instance.items())

first_instance_key_id = next(iter(dev_key))
first_key = {first_instance_id: dev_key[first_instance_key_id]}
print(first_key.items())

# Test the yarowsky_algorithm function
single_yarowsky_algorithm_prediction = yarowsky_algorithm(first_instance, first_key, k=10, iterations=10)
print("Single Instance yarowsky_algorithm Prediction:", single_yarowsky_algorithm_prediction)


dict_items([('d001.s001.t002', <__main__.WSDInstance object at 0x7dd58196edd0>)])
dict_items([('d001.s001.t002', ['group%1:03:00::'])])
Single Instance yarowsky_algorithm Prediction: {'d001.s001.t002': 'group%1:03:00::'}


**Hyperparameter tuning**

In [ ]:
def yarowsky_algorithm_with_params(dev_instances, dev_key, k=10, iterations=10, window_size=None):
    '''
    Yarowsky's algorithm with adjustable hyperparameters.
    '''
    # Initialize seeds
    seeds = {}
    for my_id, instance in dev_instances.items():
        if my_id in dev_key:
            correct_senses = dev_key[my_id]
            for sense in correct_senses:
                if sense not in seeds:
                    seeds[sense] = set()
                seeds[sense].add(my_id)

    # Iterative sense propagation
    for iteration in range(iterations):
        new_seeds = {sense: set(seeds[sense]) for sense in seeds}
        for my_id, instance in dev_instances.items():
            if my_id in dev_key:
                continue
            context = set(instance.context[:window_size]) if window_size else set(instance.context)

            # Find best matching sense
            best_sense = None
            best_overlap = 0
            for sense, seed_instances in seeds.items():
                overlap = len(context.intersection(*[set(dev_instances[id].context[:window_size])
                                                     for id in seed_instances if id in dev_instances]))
                if overlap > best_overlap:
                    best_sense = sense
                    best_overlap = overlap

            if best_sense:
                new_seeds[best_sense].add(my_id)

        seeds = new_seeds

    # Predict senses
    predicted_senses = {}
    for my_id, instance in dev_instances.items():
        best_sense = None
        best_overlap = 0
        context = set(instance.context[:window_size]) if window_size else set(instance.context)
        for sense, seed_instances in seeds.items():
            overlap = len(context.intersection(*[set(dev_instances[id].context[:window_size])
                                                 for id in seed_instances if id in dev_instances]))
            if overlap > best_overlap:
                best_sense = sense
                best_overlap = overlap
        predicted_senses[my_id] = best_sense

    return predicted_senses

def hyperparameter_tuning(dev_instances, dev_key, k_values, iterations_values, window_sizes):
    '''
    Perform hyperparameter tuning using grid search.
    '''
    best_accuracy = 0
    best_params = None

    for k in k_values:
        for iterations in iterations_values:
            for window_size in window_sizes:
                predicted_senses = yarowsky_algorithm_with_params(dev_instances, dev_key, k=k, iterations=iterations, window_size=window_size)
                accuracy = calculate_accuracy(predicted_senses, dev_key)
                print(f'k={k}, iterations={iterations}, window_size={window_size}, Accuracy={accuracy * 100:.2f}%')

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = (k, iterations, window_size)

    print(f'Best Parameters: k={best_params[0]}, iterations={best_params[1]}, window_size={best_params[2]}')
    print(f'Best Development Accuracy: {best_accuracy * 100:.2f}%')
    return best_params

if __name__ == '__main__':

    # Define hyperparameter search space
    k_values = [5, 10, 20]
    iterations_values = [5, 10, 15]
    window_sizes = [5, 10, None]  # None means full context

    # Perform hyperparameter tuning
    best_params = hyperparameter_tuning(dev_instances, dev_key, k_values, iterations_values, window_sizes)


k=5, iterations=5, window_size=5, Accuracy=14.43%
k=5, iterations=5, window_size=10, Accuracy=14.43%
k=5, iterations=5, window_size=None, Accuracy=14.43%
k=5, iterations=10, window_size=5, Accuracy=14.43%
k=5, iterations=10, window_size=10, Accuracy=14.43%
k=5, iterations=10, window_size=None, Accuracy=14.43%
k=5, iterations=15, window_size=5, Accuracy=14.43%
k=5, iterations=15, window_size=10, Accuracy=14.43%
k=5, iterations=15, window_size=None, Accuracy=14.43%
k=10, iterations=5, window_size=5, Accuracy=14.43%
k=10, iterations=5, window_size=10, Accuracy=14.43%
k=10, iterations=5, window_size=None, Accuracy=14.43%
k=10, iterations=10, window_size=5, Accuracy=14.43%
k=10, iterations=10, window_size=10, Accuracy=14.43%
k=10, iterations=10, window_size=None, Accuracy=14.43%
k=10, iterations=15, window_size=5, Accuracy=14.43%
k=10, iterations=15, window_size=10, Accuracy=14.43%
k=10, iterations=15, window_size=None, Accuracy=14.43%
k=20, iterations=5, window_size=5, Accuracy=14.43%
k=20

In [ ]:
# Best hyperparameters from tuning
best_k = 5
best_iterations = 5
best_window_size = 5

# Apply Yarowsky's algorithm to the test set using the best parameters
predicted_test_senses = yarowsky_algorithm_with_params(
    test_instances, test_key,
    k=best_k,
    iterations=best_iterations,
    window_size=best_window_size
)

# Calculate and print test accuracy
test_accuracy = calculate_accuracy(predicted_test_senses, test_key)
print(f'Test Accuracy using Best Hyperparameters: {test_accuracy * 100:.2f}%')


Test Accuracy using Best Hyperparameters: 14.00%
